In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

import pandas as pd

import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
#from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.stem.snowball import SnowballStemmer

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/kerry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
replace_str1 = '<span class="css-1egxyvc" data-font-weight="bold">croissant</span>'
replace_str2 = '<span class="css-1egxyvc" data-font-weight="bold">croissants</span>'

In [4]:
reviews = []

with open("../archive/reviews.txt", "r") as the_file:
    while True:
        line = the_file.readline()
        if line == "":
            break
        elif len(line) > 3: # \n
            line = line.replace(replace_str1, "croissant")
            line = line.replace(replace_str2, "croissant")
            reviews.append(line)

print(len(reviews))

90


In [5]:
print(reviews[1:4])

['lost larson bakery : These are the best cinnamon rolls ever. The croissant are also delicious and the service is very efficient”\n', 'bang bang pie and biscuits : reminded me of a croissant) and light. The actual filling was very good and you can clearly taste”\n', 'good ambler : minutes. - Ham and cheese croissant: croissant was super flaky and the sesame seeds were a nice touch”\n']


In [6]:
# I like this version better but pandas words differently, so later we conform
dict_reviews = {review.split(':')[0]: review.split(':')[1] for review in reviews}

names = list(dict_reviews.keys())
words = list(dict_reviews.values())
index = ['idx'+str(idx) for idx in range(0, len(names))]
dict_for_pd = {'index': index, 'BakeryName': names, 'Review': words}

df = pd.DataFrame.from_dict(dict_for_pd)
df.set_index("index", inplace = True)
df.head()

In [26]:
# some peering
pd.options.display.max_colwidth = None
df[df['Review'].str.contains("flak")]

,BakeryName,Review,Review_Prepped
index,,,
idx16,hoosier mama pie company,and cream admixture and a flaky buttery crust almost like that of a croissant. Indecisive and gluttonous as”\n,"[cream, admixtur, flaki, butteri, crust, almost, like, croissant, ., indecis, glutton, ”]"
idx33,caffe umbria,"are the highlight of the show, you absolutely have to get a croissant - it was so flaky and buttery and the muffin was also moist and had a hint of cinnamon!!!”\n","[highlight, show, ,, absolut, get, croissant, -, flaki, butteri, muffin, also, moist, hint, cinnamon, !!!”]"
idx39,"abc bakery and deli norridge?hrid=SDxAypM6HgjzV47KHlp0TQ&amp;osq=croissant"">more</a></span></p","of the seductive sweets. I chose a nutella croissant! Fluffy, flaky, sticky hazelnut spread glory. Yes”\n","[seduct, sweet, ., chose, nutella, croissant, !, fluffi, ,, flaki, ,, sticki, hazelnut, spread, glori, ., yes, ”]"


In [ ]:
# hoosier mama is not a review on a croissant
# not a huge nutella fan, it's just sugar. too easy
# caffe umbria is the winner based on this.
# but let's do more with these reviews
# while we r here. 

In [27]:
df[df['Review'].str.contains("artisan")]

,BakeryName,Review,Review_Prepped
index,,,


In [8]:
# tokenizing, filtering stop words, stemming, and lower casing

stop_words = set(stopwords.words("english"))
stemmer = SnowballStemmer('english')

In [12]:
df['Review_Prepped'] = df.Review.apply(lambda xyz: 
                                       [stemmer.stem(word.lower()) 
                                        for word in wordpunct_tokenize(xyz) 
                                        if word.lower() not in stop_words])

In [16]:
df.head()

,BakeryName,Review,Review_Prepped
index,,,
idx0,hendrickx belgian bread crafter,The almond chocolate croissant and french cou...,"[almond, chocol, croissant, french, countri, b..."
idx1,lost larson bakery,These are the best cinnamon rolls ever. The c...,"[best, cinnamon, roll, ever, ., croissant, als..."
idx2,bang bang pie and biscuits,reminded me of a croissant) and light. The ac...,"[remind, croissant, ), light, ., actual, fill,..."
idx3,good ambler,minutes. - Ham and cheese croissant,"[minut, ., -, ham, chees, croissant]"
idx4,p%C3%A2tisserie coralie,is a croissant desert. Search no more. Very f...,"[croissant, desert, ., search, ., friend, staf..."


In [11]:
# semantic analysis